In [1]:
import chess
import chess.pgn
from io import StringIO
import stockfish
import re
import time

In [10]:
games = 0
game = []
after_moves = False
after_pgn = False
parsed_games = []
firstline = True
sf = stockfish.Stockfish('./stockfish-10-win/Windows/stockfish_10_x64.exe',
                         depth=18)
with open('F:/lichess/august_games.pgn', 'r') as f:
    for line in f:
        game.append(line)
        if line == '\n':
            if after_moves:
                after_pgn = True
                after_moves = False
            else:
                after_moves = True
        if after_pgn:
            parsed_game = chess.pgn.read_game(StringIO(''.join(game)))
            game = []
            games += 1
            after_pgn = False
            
            fen = parsed_game.end().board().fen()
            end = parsed_game.headers['Termination']
            game_link = parsed_game.headers['Site']
            black = parsed_game.headers['Black']
            white = parsed_game.headers['White']
            end = parsed_game.headers['Termination']
            
            sf.set_fen_position(fen)
            sf.get_best_move()
            info_string = sf.info
            
            rating_match = re.search(r'score (cp|mate) (.+?)(?: |$)',info_string)
            if rating_match.group(1) == 'mate':
                original_rating = int(rating_match.group(2))
                if original_rating:
                    rating = 9999 * original_rating / abs(original_rating)
                elif parsed_game.headers['Result'] == '1-0':
                    rating = 9999
                else:
                    rating = -9999
            else:
                rating = int(rating_match.group(2))
            if ' b ' in fen:
                rating *= -1
            print(game_link, white, black, rating, end)
        if games >= 4:
            break

Headers(Event='Rated Bullet tournament https://lichess.org/tournament/wqEDRBia', Site='https://lichess.org/EeKb62cG', Date='2019.08.01', Round='-', White='Karnak13', Black='Sergiomeira8', Result='1-0', BlackElo='1534', BlackRatingDiff='-5', ECO='D31', Opening='Semi-Slav Defense: Accelerated Move Order', Termination='Time forfeit', TimeControl='120+1', UTCDate='2019.08.01', UTCTime='00:00:00', WhiteElo='1752', WhiteRatingDiff='+5')
score cp 1144 
-1144 Time forfeit
Headers(Event='Rated Bullet tournament https://lichess.org/tournament/wqEDRBia', Site='https://lichess.org/y1vpWdnP', Date='2019.08.01', Round='-', White='Princepe28', Black='HojeNao', Result='0-1', BlackElo='1861', BlackRatingDiff='+9', ECO='C00', Opening='French Defense: Two Knights Variation', Termination='Normal', TimeControl='120+1', UTCDate='2019.08.01', UTCTime='00:00:00', WhiteElo='1832', WhiteRatingDiff='-9')
score cp -1152 
-1152 Normal
Headers(Event='Rated Blitz game', Site='https://lichess.org/FDuEtjq7', Date='201

In [48]:
sf = stockfish.Stockfish('./stockfish-10-win/Windows/stockfish_10_x64.exe',
                         depth=22)

In [7]:
sf.set_fen_position('r3k3/pp3pb1/8/2p1P3/2Pp1pN1/1P1P4/3NRP2/2q1K3 w q - 1 26')

In [52]:
sf.set_position(['e2e4'])

In [8]:
sf.get_best_move()
sf.info

'info depth 18 seldepth 15 multipv 1 score cp 0 nodes 2058685 nps 2462541 tbhits 0 time 836 pv h5e5 f7f6 b2b4 d6e6 e5d4 g7f7 d4a7 f7f8 b4b5 f6g6 g2f3 g6f6 f3g3 f6g6'

In [60]:
info_str = 'info depth 22 seldepth 36 multipv 1 score cp 1493 nodes 552164 nps 3119570 tbhits 0 time 177 pv b5b4 b7f7 g7f7 g3f4 g5g4 h3g4 b4b3 f4e5 b3b2 e5d5 b2b1q d5e5 b1d3 d4d5 d3e3 d5d6 e3d3 e5f4 e4e3 f4f3 e3e2 f3f2 d3e4 f2e1 e4g4'

In [64]:
match = re.search(r'score (cp|mate) (.+?) ',info_str)

In [65]:
match.group(1)

'1493'